In [29]:
import requests
from bs4 import BeautifulSoup

In [30]:
def extract(page):
    print(page)
    url = f'https://www.yell.com/ucs/UcsSearchAction.do?keywords=coffee&location=London&scrambleSeed=843665584&pageNum={page}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html = response.text
    return BeautifulSoup(html, 'html.parser')

In [31]:
def translate(soup):
    articles = soup.find_all('div', {'class': 'row businessCapsule--mainRow'})
    for article in articles:
        name = article.find('h2', {'class': 'businessCapsule--name text-h2'}).text
        short_description = article.find('span', {'class': 'businessCapsule--classification'})
        website = article.find('a', {'class': 'btn btn-yellow businessCapsule--ctaItem'})
        phone = article.find('span', {'class': 'business--telephoneNumber'})
        map_link = article.find('a', {'class': 'col-sm-24 businessCapsule--address businessCapsule--link'})
        address = article.find('span', {'itemprop': 'address'})
        features = map(lambda feature : feature.text.strip('● '), 
                       article.find_all('li', {'class': 'col-sm-12 businessCapsule--servicesItem'}))
        rating = article.find('span', {'class': 'starRating--average'})
        reviews = article.find('span', {'class': 'starRating--total'})
        
        shop_info = dict(
            name = name,
            short_description = (short_description or '') and short_description.text,
            website = (website or '') and website['href'],
            phone = (phone or '') and phone.text,
            map_link = (map_link or '') and map_link['href'],
            address = (address or '') and address.text.replace('\n', ''),
            features = ', '.join(features),
            rating = (rating or '') and rating.text.strip(),
            reviews = (reviews or '') and reviews.text.strip('(').strip(')')
        )
        yield shop_info

In [32]:
import pandas as pd
from itertools import chain

In [33]:
def save_to_file(filename, pages = 10):
    full_data = chain.from_iterable(translate(extract(page)) for page in range(1, pages+1))
    df = pd.DataFrame(full_data)
    df.to_csv(filename)

In [34]:
save_to_file('coffe_london.csv')

1
2
3
4
5
6
7
8
9
10
